In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit.extensions import UnitaryGate
from qiskit import Aer

In [2]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath('../classifiers'))
from classifiers_ICQ import iqc_classifier

In [3]:
def get_prob_1(hist):
    count0 = 0
    count1 = 0
    if '0' in hist:
        count0 = hist['0']
    if '1' in hist:
        count1 = hist['1']
    return count1/(count1+count0)

In [4]:
dic_classifier_params = {}
dic_classifier_params["sigma_q_params"] = [1,1,1,1]
dic_classifier_params["use_polar_coordinates_on_sigma_q"] = False
dic_classifier_params["load_inputvector_env_state"] = False

In [5]:
# Simulation in numpy

# Since we may have multiple environments, vectorW is now a array of arrays...
vectorX = [0.91, 0.90]
vectorW = [[0.919, 0.491]]
z, p_cog, qo = iqc_classifier(vector_x=vectorX, vector_ws=vectorW, dic_classifier_params=dic_classifier_params)

# ... and we need to pick up the first (and only, since on this experiment we have only 1 environment) quantum operator
qo = qo[0]

print("prob 1 in the simulation", p_cog)

# Simulation in qiskit (O AMBIENTE É O PRIMEIRO QUBIT E COG É O SEGUNDO QUBIT)
matrix = qo
gate = UnitaryGate(matrix)

# Creating circuit that will contain our transpiled quantum operator
circuit = QuantumCircuit(2,1)
circuit.append(gate, [0, 1])
trans_circuit = transpile(circuit, basis_gates=['cx', 'u3'])

# Creating the circuit which will contain the whole operation
circuit_new = QuantumCircuit(2,1)
circuit_new.h(0)
circuit_new.h(1)
circuit_new = circuit_new.compose(trans_circuit, [0,1])

# Executing the simulation
circuit_new.measure(1,0)
aer_sim = Aer.get_backend('aer_simulator')
job = aer_sim.run(circuit_new, shots=10000)
hist = job.result().get_counts()

print("prob 1 in the circuit", get_prob_1(hist))

# Drawing circuit
print(circuit_new.draw())

prob 1 in the simulation (0.1603325576368378+0j)
prob 1 in the circuit 0.1599
global phase: 3.481
     ┌───┐       ┌─────────────┐                  ┌────────────────┐          »
q_0: ┤ H ├───────┤ U3(π,π/2,0) ├──────────■───────┤ U3(π,π/4,-π/4) ├───────■──»
     ├───┤┌──────┴─────────────┴───────┐┌─┴─┐┌────┴────────────────┴────┐┌─┴─┐»
q_1: ┤ H ├┤ U3(0.81176,0.22565,1.2488) ├┤ X ├┤ U3(1.5421,-1.317,1.7945) ├┤ X ├»
     └───┘└────────────────────────────┘└───┘└──────────────────────────┘└───┘»
c: 1/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«      ┌────────────────────────┐    
«q_0: ─┤ U3(0,-0.28395,0.08675) ├────
«     ┌┴────────────────────────┴┐┌─┐
«q_1: ┤ U3(2.3898,2.9842,3.0753) ├┤M├
«     └──────────────────────────┘└╥┘
«c: 1/═════════════════════════════╩═
«                                  0 


In [6]:
# Simulation in numpy

# Since we may have multiple environments, vectorW is now a array of arrays...
vectorX = [0.91, 0.90, 0.3, 0.5]
vectorW = [[0.919, 0.491, 0.4, 0.5]]
z, p_cog, qo = iqc_classifier(vector_x=vectorX, vector_ws=vectorW, dic_classifier_params=dic_classifier_params)

# ... and we need to pick up the first (and only, since on this experiment we have only 1 environment) quantum operator
qo = qo[0]

print("prob 1 in the simulation", p_cog)

# Simulation in qiskit (O AMBIENTE É O PRIMEIRO QUBIT E COG É O SEGUNDO QUBIT)
matrix = qo
gate = UnitaryGate(matrix)

# Creating circuit that will contain our transpiled quantum operator
circuit = QuantumCircuit(3,1)
circuit.append(gate, [0, 1, 2])
trans_circuit = transpile(circuit, basis_gates=['cx', 'u3'])

# Creating the circuit which will contain the whole operation
circuit_new = QuantumCircuit(3,1)
circuit_new.h(0)
circuit_new.h(1)
circuit_new.h(2)
circuit_new = circuit_new.compose(trans_circuit, [0, 1, 2])

# Executing the simulation
circuit_new.measure(1,0)
aer_sim = Aer.get_backend('aer_simulator')
job = aer_sim.run(circuit_new, shots=10000)
hist = job.result().get_counts()

print("prob 1 in the circuit", get_prob_1(hist))

# Drawing circuit
print(circuit_new.draw())

prob 1 in the simulation (0.2802491340693895+0j)
prob 1 in the circuit 0.4917
global phase: 2.1291
     ┌───┐  ┌───────────────────────┐       ┌─────────────────────┐     »
q_0: ┤ H ├──┤ U3(π/2,-π/2,0.044418) ├────■──┤ U3(π/2,3.0641,-π/2) ├──■──»
     ├───┤  └─┬────────────────────┬┘  ┌─┴─┐└┬────────────────────┤  │  »
q_1: ┤ H ├────┤ U3(π/2,-π,0.84711) ├───┤ X ├─┤ U3(π/2,0.71858,-π) ├──┼──»
     ├───┤┌───┴────────────────────┴──┐└───┘ └────────────────────┘┌─┴─┐»
q_2: ┤ H ├┤ U3(0,0,-2.57794669643175) ├────────────────────────────┤ X ├»
     └───┘└───────────────────────────┘                            └───┘»
c: 1/═══════════════════════════════════════════════════════════════════»
                                                                        »
«                                                                            »
«q_0: ────────────────────────────────────────────────────────────────────■──»
«                                                                         │  